
<img alt="Mepylome Logo" src="https://raw.githubusercontent.com/brj0/mepylome/main/mepylome/data/assets/mepylome.svg" width="300">

Mepylome: A Toolkit for DNA-Methylation Analysis in Tumor Diagnostics
=====================================================================

This notebook automates the analysis outlined in the Mepylome publication,
performing all necessary steps from downloading datasets to executing the
analyses.


### Usage

- All datasets and outputs are saved in `~/mepylome`.
- Follow the notebook/script step-by-step for an in-depth understanding of
  the workflow and results.


### System Requirements

- *Operating System*: Ubuntu 20.04.6
- *Python Version*: 3.12


### Reference Publication (will follow)

- *Title*: Mepylome: A User-Friendly Open-Source Toolkit for DNA-Methylation
  Analysis in Tumor Diagnostics
- *Author*: Jon Brugger et al.


### Run This Notebook in Google Colab

You can quickly open and run this notebook in Google Colab without any setup
by clicking the link below.

**Note**: The graphical user interface (GUI) features won't run in Colab, but
the rest of the analysis will work as expected.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/brj0/mepylome/blob/main/examples/publication_analysis.ipynb)


This notebook was automatically generated from the corresponding py-file
with:

```bash
jupytext --to ipynb publication_analysis.py
```


-----------------------------------------------------------------------------

## Contents
0. **[Initialization](#0.-Initialization)**
1. **[Salivary Gland Tumors](#1.-Salivary-Gland-Tumors)**
2. **[Soft Tissue Tumors](#2.-Soft-Tissue-Tumors)**
3. **[Squamous Cell Carcinoma](#3.-Squamous-Cell-Carcinoma)**
4. **[Appendix](#4.-Appendix)**

-----------------------------------------------------------------------------
<a name="0.-Initialization"></a>
## 0. Initialization

### Install Required Packages

To run the analysis, install the following Python packages:
- `mepylome` – the main toolkit for DNA-methylation analysis.
- `linear_segment` – used for segmentation calculations in CNV plots.
- `kaleido` for saving plots.

Install these packages using the commands below:

In [ ]:
%%bash
echo "Install necessary packages. This may take 1 to 2 minutes..."

pip install mepylome
pip install ipython
pip install pillow
pip install linear_segment
pip install -U kaleido

echo
echo "Installation complete."

### Core Imports, Configuration and main Functions

In [ ]:
import io
import json
import os
import shutil
import subprocess
import tarfile
import zipfile
from pathlib import Path

import numpy as np
import pandas as pd
import requests
from IPython.display import Image as IPImage
from PIL import Image

from mepylome import ArrayType, Manifest
from mepylome.analysis import MethylAnalysis
from mepylome.dtypes.manifests import (
    DOWNLOAD_DIR,
    MANIFEST_URL,
    REMOTE_FILENAME,
)
from mepylome.utils import ensure_directory_exists
from mepylome.utils.files import (
    download_file,
    download_geo_probes,
)

# Define output font size for plots
FONTSIZE = 23
GEO_URL = "https://www.ncbi.nlm.nih.gov/geo/download/?acc={acc}&format=file"

# Define dataset URLs and filenames
datasets = {
    "salivary_gland_tumors": {
        "xlsx": "https://ars.els-cdn.com/content/image/1-s2.0-S0893395224002059-mmc4.xlsx",
        "geo_ids": ["GSE243075"],
    },
    "soft_tissue_tumors": {
        "xlsx": "https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-020-20603-4/MediaObjects/41467_2020_20603_MOESM4_ESM.xlsx",
        "geo_ids": ["GSE140686"],
    },
    "sinonasal_tumors": {
        "xlsx": "https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-022-34815-3/MediaObjects/41467_2022_34815_MOESM6_ESM.xlsx",
        "geo_ids": ["GSE196228"],
    },
    "scc": {
        "xlsx": "https://www.science.org/doi/suppl/10.1126/scitranslmed.aaw8513/suppl_file/aaw8513_data_file_s1.xlsx",
        "geo_ids": ["GSE85566"],
    },
    "scc_test": {
        "geo_ids": [
            "GSE124052",
            "GSE66836",
            "GSE79556",
            "GSE87053",
            "GSE95036",
            "GSE124052",
        ],
    },
}

# Determine basic storage directory depending on platform
if "COLAB_GPU" in os.environ:
    # Google Colab
    mepylome_dir = Path("/content/mepylome")
elif os.path.exists("/mnt/bender"):
    # Bender-specific path
    mepylome_dir = Path("/mnt/bender/mepylome")
else:
    # Default for local Linux or other environments
    mepylome_dir = Path.home() / "mepylome"

# Ensure the directory exists
os.makedirs(mepylome_dir, exist_ok=True)
os.environ["MEPYLOME_DIR"] = str(mepylome_dir)
print(f"Data will be stored in: {mepylome_dir}")

data_dir = mepylome_dir / "data"
output_dir = mepylome_dir / "out"
reference_dir = mepylome_dir / "cn_neutral_idats"
validation_dir = mepylome_dir / "validation_data"

ensure_directory_exists(data_dir)
ensure_directory_exists(output_dir)
ensure_directory_exists(reference_dir)
ensure_directory_exists(validation_dir)

# Main Funktions


def extract_tar(tar_path, output_directory):
    """Extracts tar file under 'tar_path' to 'output_directory'."""
    ensure_directory_exists(output_directory)
    with tarfile.open(tar_path, "r") as tar:
        tar.extractall(path=output_directory, filter="data")
        print(f"Extracted {tar_path} to {output_directory}")


def download_from_geo_and_untar(analysis_dir, geo_ids):
    """Downloads all missing GEO files and untars them."""
    for geo_id in geo_ids:
        idat_dir = analysis_dir / geo_id
        if idat_dir.exists():
            print(f"Data for GEO ID {geo_id} already exists, skipping.")
            continue
        try:
            tar_path = analysis_dir / f"{geo_id}.tar"
            geo_url = GEO_URL.format(acc=geo_id)
            download_file(geo_url, tar_path)
            extract_tar(tar_path, idat_dir)
            tar_path.unlink()
        except Exception as e:
            print(f"Error processing GEO ID {geo_id}: {e}")


def calculate_cn_summary(analysis, class_):
    """Calculates and saves CN summary plots."""
    df_class = analysis.idat_handler.samples_annotated[class_]
    plot_list = []
    analysis_dir = analysis.analysis_dir
    all_classes = sorted(df_class.unique())
    for methyl_class in all_classes:
        df_index = df_class == methyl_class
        sample_ids = df_class.index[df_index]
        plot, df_cn_summary = analysis.cn_summary(sample_ids)
        plot.update_layout(
            title=f"{methyl_class}",
            title_x=0.5,
            yaxis_title="Proportion of CNV gains/losses",
        )
        plot.update_layout(
            title_font_size=FONTSIZE + 3,
            yaxis_title_font_size=FONTSIZE - 2,
        )
        plot_list.append(plot)
    png_paths = [
        output_dir / f"{analysis_dir.name}-cn_summary-{x}.png"
        for x in all_classes
    ]
    for path, fig in zip(png_paths, plot_list):
        fig.write_image(path)
    images = [Image.open(path) for path in png_paths]
    width, height = images[0].size
    n_columns = 4
    n_images = len(images)
    n_rows = (n_images + n_columns - 1) // n_columns
    total_width = width * n_columns
    total_height = height * n_rows
    new_image = Image.new("RGB", (total_width, total_height), (255, 255, 255))
    for index, img in enumerate(images):
        row = index // n_columns
        col = index % n_columns
        x = col * width
        y = row * height
        new_image.paste(img, (x, y))
    output_path = output_dir / f"{analysis_dir.name}-cn_summary.png"
    new_image.save(output_path)
    return output_path

### Blacklist Generation for CpG Sites

Some CpG sites should be excluded from the analysis. Here we choose probes
flagged with `MFG_Change_Flagged` that should be excluded according to the
manifest and those that are on sex chromosomes.

In [ ]:
def generate_blacklist_cpgs():
    """Returns and caches CpG sites that should be blacklisted."""
    print("Generating blacklist. Can take some time...")
    blacklist_path = data_dir / "cpg_blacklist.csv"
    if not blacklist_path.exists():
        manifest_url = MANIFEST_URL[ArrayType.ILLUMINA_EPIC]
        ensure_directory_exists(DOWNLOAD_DIR)
        response = requests.get(manifest_url)
        html_sucess_ok_code = 200
        if response.status_code == html_sucess_ok_code:
            with zipfile.ZipFile(io.BytesIO(response.content)) as thezip:
                thezip.extractall(DOWNLOAD_DIR)
        else:
            raise Exception(
                f"Failed to download the file: {response.status_code}"
            )
        csv_path = DOWNLOAD_DIR / REMOTE_FILENAME[ArrayType.ILLUMINA_EPIC]
        manifest_df = pd.read_csv(csv_path, skiprows=7)
        flagged_cpgs = manifest_df[
            manifest_df["MFG_Change_Flagged"].fillna(False)
        ]["IlmnID"]
        flagged_cpgs.to_csv(blacklist_path, index=False, header=False)
        csv_path.unlink()
    blacklist = pd.read_csv(blacklist_path, header=None)
    print("Generating blacklist done.")
    return set(blacklist.iloc[:, 0])


def sex_chromosome_cpgs():
    """Returns CpGs on sex chromosomes for EPIC and 450k arrays."""
    manifest = Manifest("epic")
    sex_cpgs_epic = manifest.data_frame[
        manifest.data_frame.Chromosome.isin([23, 24])
    ].IlmnID
    manifest = Manifest("450k")
    sex_cpgs_450k = manifest.data_frame[
        manifest.data_frame.Chromosome.isin([23, 24])
    ].IlmnID
    return set(sex_cpgs_epic) | set(sex_cpgs_450k)


# Choose CpG list that should be blacklisted
blacklist = generate_blacklist_cpgs() | sex_chromosome_cpgs()

### Copy-Neutral Reference Probes

To ensure accurate analysis, we utilize control probes from the [Koelsche et
al. (2021) study](https://doi.org/10.1038/s41467-020-20603-4). These probes
are stored in the designated reference directory `reference_dir`.

**Best Practices**:
- Include both fresh-frozen and FFPE (formalin-fixed paraffin-embedded)
  samples in the copy-neutral reference set for optimal results.

In [ ]:
cn_neutral_probes = [
    "GSM4180453_201904410008_R06C01",
    "GSM4180454_201904410008_R05C01",
    "GSM4180455_201904410008_R04C01",
    "GSM4180456_201904410008_R03C01",
    "GSM4180457_201904410008_R02C01",
    "GSM4180458_201904410008_R01C01",
    "GSM4180459_201904410007_R08C01",
    "GSM4180460_201904410007_R07C01",
    "GSM4180741_201247480004_R05C01",
    "GSM4180742_201247480004_R04C01",
    "GSM4180743_201247480004_R03C01",
    "GSM4180751_201194010006_R01C01",
    "GSM4180909_200394870074_R04C02",
    "GSM4180910_200394870074_R03C02",
    "GSM4180911_200394870074_R02C02",
    "GSM4180912_200394870074_R01C02",
    "GSM4180913_200394870074_R05C01",
    "GSM4180914_200394870074_R04C01",
    "GSM4181456_203049640041_R03C01",
    "GSM4181509_203049640040_R07C01",
    "GSM4181510_203049640040_R08C01",
    "GSM4181511_203049640041_R01C01",
    "GSM4181512_203049640041_R02C01",
    "GSM4181513_203049640041_R04C01",
    "GSM4181514_203049640041_R05C01",
    "GSM4181515_203049640041_R06C01",
    "GSM4181516_203049640041_R07C01",
    "GSM4181517_203049640041_R08C01",
]

download_geo_probes(reference_dir, cn_neutral_probes)

-----------------------------------------------------------------------------
<a name="1.-Salivary-Gland-Tumors"></a>
## 1. Salivary Gland Tumors

This section replicates the methylation analysis performed in the study by
[Jurmeister et al. (2024)](https://doi.org/10.1016/j.modpat.2024.100625). To
begin, we download the required data and organize it within the designated
directories.

In [ ]:
# Initialize directories.
tumor_site = "salivary_gland_tumors"
analysis_dir_sg = data_dir / tumor_site
test_dir_sg = validation_dir / tumor_site

ensure_directory_exists(test_dir_sg)
ensure_directory_exists(analysis_dir_sg)

# Download the the annotation spreadsheet.
if not (
    excel_path := analysis_dir_sg / f"{tumor_site}-annotation.xlsx"
).exists():
    download_file(datasets[tumor_site]["xlsx"], excel_path)
    # Deletes the first 2 rows (useless description).
    pd.read_excel(excel_path, skiprows=2).to_excel(excel_path, index=False)

# Download the IDAT files.
download_from_geo_and_untar(analysis_dir_sg, datasets[tumor_site]["geo_ids"])

### Create the Methylation Analysis Object

The `MethylAnalysis` object serves as the main interface for performing DNA
methylation analysis. Key parameters such as the directory structure, number
of CpG sites, and UMAP settings are configured here.

In [ ]:
analysis_sg = MethylAnalysis(
    analysis_dir=analysis_dir_sg,
    reference_dir=reference_dir,
    output_dir=output_dir,
    test_dir=test_dir_sg,
    n_cpgs=25000,
    load_full_betas=True,
    overlap=False,
    cpg_blacklist=blacklist,
    debug=True,
    do_seg=True,
    umap_parms={
        "n_neighbors": 8,
        "metric": "manhattan",
        "min_dist": 0.3,
    },
)

### Load Beta Values

Reads and processes beta values from the provided dataset. This step can also
be performed interactively within the GUI.

In [ ]:
analysis_sg.set_betas()

### Generate UMAP Plot

Set the columns used for coloring the UMAP plot before initiating the
dimensionality reduction process. The UMAP algorithm produces a visual
representation of the sample clusters, which is stored as a Plotly object in
`analysis_sg.umap_plot`.

In [ ]:
# Calculate UMAP
analysis_sg.idat_handler.selected_columns = ["Methylation class"]
analysis_sg.make_umap()

In [ ]:
# Show the results
print(analysis_sg.umap_df)
output_path = output_dir / f"{analysis_dir_sg.name}-umap_plot.jpg"
analysis_sg.umap_plot.write_image(
    output_path,
    format="jpg",
    width=2000,
    height=1000,
    scale=2,
)
IPImage(filename=output_path)

### Launch the Analysis GUI

Initializes an interactive GUI for further exploration of the methylation
data.

**Note**: This step is only supported in local environments (not in
cloud-based platforms like Google Colab or Binder).

In [ ]:
analysis_sg.run_app(open_tab=True)

### Generate and Save CNV Plot

Creates a copy number variation (CNV) plot for a specified sample and saves
the output as a high-resolution image.

In [ ]:
# Save CNV example
analysis_sg.make_cnv_plot("206842050057_R06C01")
cnv_plot = analysis_sg.cnv_plot
cnv_plot.update_layout(
    yaxis_range=[-1.1, 1.1],
    font={"size": FONTSIZE},
    margin={"t": 50},
)
output_path = output_dir / f"{analysis_dir_sg.name}-cnv_plot.jpg"
cnv_plot.write_image(
    output_path,
    format="jpg",
    width=2000,
    height=1000,
    scale=2,
)
IPImage(filename=output_path)

### Generate CNV Summary Plots

In addition to individual CNV plots, this step computes summary plots to
highlight genomic alterations across multiple samples.

**Note**:
Generating all copy number variation (CNV) plots can be resource-intensive.
The process may take up to 30 minutes, depending on the computational
resources available.

In [ ]:
analysis_sg.precompute_cnvs()
cn_summary_path_sg = calculate_cn_summary(analysis_sg, "Methylation class")

In [ ]:
IPImage(filename=cn_summary_path_sg)

### Supervised Classifier Validation

The next step involves validating various supervised classification
algorithms to evaluate their performance on the dataset. This process helps
identify the most accurate model for methylation-based classification.

**Note**:
Training can be resource-intensive. The process may take up to 10 minutes,
depending on the computational resources available.

In [ ]:
# Train supervised classifiers
ids = analysis_sg.idat_handler.ids
clf_out_sg = analysis_sg.classify(
    ids=ids,
    clf_list=[
        "none-kbest-et",
        "none-kbest-lr",
        "none-kbest-rf",
        "none-kbest-svc_rbf",
        "none-pca-lr",
        "none-pca-et",
        "none-none-knn",
    ],
)

In [ ]:
# Print reports for all classifier for the first sample
for clf_result in clf_out_sg:
    print(clf_result.reports[0])

In [ ]:
# Identify and display the best classifier
best_clf_sg = max(
    clf_out_sg, key=lambda result: np.mean(result.metrics["accuracy_scores"])
)
print("Most accurate classifier:")
print(best_clf_sg.reports[0])

-----------------------------------------------------------------------------
<a name="2.-Soft-Tissue-Tumors"></a>
## 2. Soft Tissue Tumors

This section replicates the methylation analysis performed in the study by
[Koelsche2021 study](https://doi.org/10.1038/s41467-020-20603-4). To begin,
we download the required data and organize it within the designated
directories.

In [ ]:
# Initialize directories.
tumor_site = "soft_tissue_tumors"
analysis_dir_sf = data_dir / tumor_site
test_dir_sf = validation_dir / tumor_site

ensure_directory_exists(test_dir_sf)
ensure_directory_exists(analysis_dir_sf)

# Download the the annotation spreadsheet.
if not (
    excel_path := analysis_dir_sf / f"{tumor_site}-annotation.xlsx"
).exists():
    download_file(datasets[tumor_site]["xlsx"], excel_path)

# Download the IDAT files.
download_from_geo_and_untar(analysis_dir_sf, datasets[tumor_site]["geo_ids"])

### Create the Methylation Analysis Object

The `MethylAnalysis` object serves as the main interface for performing DNA
methylation analysis. Key parameters such as the directory structure, number
of CpG sites, and UMAP settings are configured here.

In [ ]:
analysis_sf = MethylAnalysis(
    analysis_dir=analysis_dir_sf,
    reference_dir=reference_dir,
    output_dir=output_dir,
    n_cpgs=25000,
    load_full_betas=True,
    overlap=False,
    cpg_blacklist=blacklist,
    debug=True,
    do_seg=True,
    umap_parms={
        "n_neighbors": 8,
        "metric": "manhattan",
        "min_dist": 0.3,
    },
)

### Load Beta Values

Reads and processes beta values from the provided dataset. This step can also
be performed interactively within the GUI.

In [ ]:
analysis_sf.set_betas()

### Generate UMAP Plot

Set the columns used for coloring the UMAP plot before initiating the
dimensionality reduction process. The UMAP algorithm produces a visual
representation of the sample clusters, which is stored as a Plotly object in
`analysis_sf.umap_plot`.

In [ ]:
# Calculate UMAP
analysis_sf.idat_handler.selected_columns = ["Methylation Class Name"]
analysis_sf.make_umap()

In [ ]:
# Show the results
print(analysis_sf.umap_df)
output_path = output_dir / f"{analysis_dir_sf.name}-umap_plot.jpg"
analysis_sf.umap_plot.write_image(
    output_path,
    format="jpg",
    width=2000,
    height=1000,
    scale=2,
)
IPImage(filename=output_path)

### Launch the Analysis GUI

Initializes an interactive GUI for further exploration of the methylation
data.

**Note**: This step is only supported in local environments (not in
cloud-based platforms like Google Colab or Binder).

In [ ]:
analysis_sf.run_app(open_tab=True)

### Generate and Save CNV Plot

Creates a copy number variation (CNV) plot for a specified sample and saves
the output as a high-resolution image.

In [ ]:
# Save CNV example
analysis_sf.make_cnv_plot("3999112131_R05C01")
cnv_plot = analysis_sf.cnv_plot
cnv_plot.update_layout(
    yaxis_range=[-1.1, 1.1],
    font={"size": FONTSIZE},
    margin={"t": 50},
)
output_path = output_dir / f"{analysis_dir_sf.name}-cnv_plot.jpg"
cnv_plot.write_image(
    output_path,
    format="jpg",
    width=2000,
    height=1000,
    scale=2,
)
IPImage(filename=output_path)

### Generate CNV Summary Plots

In addition to individual CNV plots, this step computes summary plots to
highlight genomic alterations across multiple samples.

**Note**:
Generating all copy number variation (CNV) plots can be resource-intensive.
The process may take up to 30 minutes, depending on the computational
resources available.

In [ ]:
analysis_sf.precompute_cnvs()
cn_summary_path_sf = calculate_cn_summary(analysis_sg, "Methylation class")

In [ ]:
IPImage(filename=cn_summary_path_sf)

### Supervised Classifier Validation

The next step involves validating various supervised classification
algorithms to evaluate their performance on the dataset. This process helps
identify the most accurate model for methylation-based classification.

**Note**:
Training can be resource-intensive. The process may take up to 10 minutes,
depending on the computational resources available.

In [ ]:
# Train supervised classifiers
ids = analysis_sf.idat_handler.ids
clf_out_sf = analysis_sf.classify(
    ids=ids,
    clf_list=[
        "none-kbest-et",
        "none-kbest-lr",
        "none-kbest-rf",
        "none-kbest-svc_rbf",
        "none-pca-lr",
        "none-pca-et",
        "none-none-knn",
    ],
)

In [ ]:
# Print reports for all classifier for the first sample
for clf_result in clf_out_sf:
    print(clf_result.reports[0])

# Identify and display the best classifier
best_clf_sf = max(
    clf_out_sf, key=lambda result: np.mean(result.metrics["accuracy_scores"])
)
print("Most accurate classifier:")
print(best_clf_sf.reports[0])

-----------------------------------------------------------------------------
<a name="3.-Squamous-Cell-Carcinoma"></a>
## 3. Squamous Cell Carcinoma

In this example, we aim to reproduce the pan-SCC classifier presented in the
study by [Jurmeister et al.
(2019)](https://doi.org/10.1126/scitranslmed.aaw8513). Our goal is to gather
data for Squamous Cell Carcinoma (SCC) from multiple sources, as outlined in
the publication, including datasets from The Cancer Genome Atlas (TCGA).
Datasets without IDAT files are omitted from the collection process.

In [ ]:
# Initialize directories.
tumor_site = "scc"
analysis_dir_scc = data_dir / tumor_site
test_dir_scc = validation_dir / tumor_site

ensure_directory_exists(test_dir_scc)
ensure_directory_exists(analysis_dir_scc)

### Step 1: Download TCGA Data
First we download the GDC client that is used for downloading data from TCGA.

In [ ]:
gdc_client_url = "https://gdc.cancer.gov/system/files/public/file/gdc-client_2.3_Ubuntu_x64-py3.8-ubuntu-20.04.zip"
gdc_client_bin = analysis_dir_scc / "gdc-client"

# Download and set up the GDC client
if not gdc_client_bin.exists():
    zip_path_0 = analysis_dir_scc / "gdc-client.zip"
    zip_path_1 = analysis_dir_scc / "gdc-client_2.3_Ubuntu_x64.zip"
    download_file(gdc_client_url, zip_path_0)
    with zipfile.ZipFile(zip_path_0, "r") as zip_file:
        zip_file.extractall(analysis_dir_scc)
    with zipfile.ZipFile(zip_path_1, "r") as zip_file:
        zip_file.extractall(analysis_dir_scc)
    zip_path_0.unlink()
    zip_path_1.unlink()
    gdc_client_bin.chmod(0o755)
    print(f"GDC client binary downloaded and set up at {gdc_client_bin}")
else:
    print(f"GDC client already exists at {gdc_client_bin}")

Now we download the complete TCGA data. **This may take several hours** due
to slow server connection speeds. It is recommended to run this process
overnight and not to abort the process to ensure it completes successfully.

In [ ]:
tcga_dir = analysis_dir_scc / "tcga_scc"
tcga_downloaded_tag = tcga_dir / ".download_complete"
tcga_anno_dir_tar = analysis_dir_scc / "tcga_annotations.tar.gz"
tcga_anno_dir = tcga_anno_dir_tar.with_suffix("").with_suffix("")

ensure_directory_exists(tcga_dir)

# Check if the TCGA annotation tar file exists and extract
if not tcga_anno_dir.exists():
    print("Setting up TCGA annotation directory...")
    # TODO download annotation to tcga_anno_dir_tar
    extract_tar(tcga_anno_dir_tar, analysis_dir_scc)
    print("Setting up TCGA annotation directory done.")

# Check if the download is complete
if not tcga_downloaded_tag.exists():
    print("Download has not been completed yet.")
    if not gdc_client_bin.exists():
        msg = f"Error: GDC client not found at {gdc_client_bin}"
        raise FileNotFoundError(msg)
    print("Downloading TCGA files. This may take some time!")
    manifest_file = next(tcga_anno_dir.glob("gdc_manifest.*txt"))
    if not manifest_file.exists():
        msg = "No TCGA manifest file found."
        raise FileNotFoundError(msg)
    print(f"Downloading TCGA data from manifest file: {manifest_file}")
    subprocess.run(
        [
            str(gdc_client_bin),
            "download",
            "--latest",
            "--manifest",
            manifest_file,
            "--dir",
            str(tcga_dir),
        ],
        check=True,
    )
    print("Download finished.")
    tcga_downloaded_tag.touch()
else:
    print("TCGA data already completely downloaded.")

Clean up by moving all IDAT files into one directory and removing array types
other than `450k` and `epic`.

In [ ]:
def move_idat_files_and_cleanup(root_dir):
    """Move all idat files one dir up and delete empty subdirectories."""
    root_dir = Path(root_dir)
    for sub_dir in root_dir.iterdir():
        if sub_dir.is_dir():
            idat_files = list(sub_dir.glob("*.idat"))
            for file in idat_files:
                destination = root_dir / file.name
                shutil.move(str(file), str(destination))
            shutil.rmtree(sub_dir)


def remove_invalid_array_types(root_dir):
    idat_files = root_dir.glob("*idat")
    valid_array_types = {ArrayType.ILLUMINA_450K, ArrayType.ILLUMINA_EPIC}
    for idat_file in idat_files:
        array_type = ArrayType.from_idat(idat_file)
        if not array_type in valid_array_types:
            print(f"Removing {idat_file.name} (Type: {array_type})")
            idat_file.unlink()


move_idat_files_and_cleanup(tcga_dir)
remove_invalid_array_types(tcga_dir)

Next we extract the TCGA annotation.

In [ ]:
def extract_tcga_case_id_dict(json_path):
    """Extracts a dictionary mapping from IDAT IDs to case IDs."""
    with open(json_path) as f:
        data = json.load(f)
    case_id_mapping = {}
    n_suffix = len("_Grn.idat")
    for item in data:
        file_name = item.get("file_name", "")[:-n_suffix]
        case_id = item.get("associated_entities", [{}])[0].get("case_id", "")
        if case_id and file_name:
            case_id_mapping[case_id] = file_name
    return case_id_mapping


json_metadata = next(tcga_anno_dir.glob("metadata.cart.*json"))
case_id_to_sample_id = extract_tcga_case_id_dict(json_metadata)

# Load the clinical data and map the case_id to the IDAT
tcga_annotation = pd.read_csv(tcga_anno_dir / "clinical.tsv", delimiter="\t")
tcga_annotation["Sample_ID"] = tcga_annotation["case_id"].map(
    case_id_to_sample_id
)
tcga_annotation = tcga_annotation.drop(columns="case_id")

# Rename columns
columns_dict = {
    "gender": "Sex",
    "age_at_index": "Age",
    "tissue_or_organ_of_origin": "Tumor_site",
    "site_of_resection_or_biopsy": "Site_of_resection_or_biopsy",
    "tumor_grade": "Tumor_grade",
    "morphology": "Morphology",
    "primary_diagnosis": "Primary_diagnosis",
    "Sample_ID": "Sample_ID",  # TODO
}
tcga_annotation = tcga_annotation.rename(columns=columns_dict)

# Standardize the 'Sex' column and convert 'Age' to numeric
tcga_annotation["Sex"] = tcga_annotation["Sex"].replace(
    {"female": "Female", "male": "Male"}
)
tcga_annotation["Age"] = pd.to_numeric(tcga_annotation["Age"], errors="coerce")

# Mark the samples that to be censored
diag_to_censor_stat = {
    "Warty carcinoma": 1,
    "Adenosquamous carcinoma": 1,
    "Squamous cell carcinoma, keratinizing, NOS": 0,
    "Papillary squamous cell carcinoma": 1,
    "Squamous cell carcinoma, spindle cell": 1,
    "Basaloid squamous cell carcinoma": 1,
    "Squamous cell carcinoma, small cell, nonkeratinizing": 1,
    "Squamous cell carcinoma, nonkeratinizing, NOS": 0,
    "Lymphoepithelial carcinoma": 1,
    "Squamous cell carcinoma, large cell, nonkeratinizing, NOS": 1,
    "Papillary carcinoma, NOS": 1,
    "Squamous cell carcinoma, NOS": 0,
}
tcga_annotation["Censor"] = tcga_annotation["Primary_diagnosis"].map(
    diag_to_censor_stat
)

# Condense the primary tumor site.
nsclc_sites = {
    "Lower lobe, lung",
    "Lung, NOS",
    "Main bronchus",
    "Middle lobe, lung",
    "Overlapping lesion of lung",
    "Upper lobe, lung",
}
hnsq_sites = {
    "Anterior floor of mouth",
    "Base of tongue, NOS",
    "Border of tongue",
    "Cheek mucosa",
    "Floor of mouth, NOS",
    "Gum, NOS",
    "Hard palate",
    "Head, face or neck, NOS",
    "Hypopharynx, NOS",
    "Larynx, NOS",
    "Lip, NOS",
    "Lower gum",
    "Mandible",
    "Mouth, NOS",
    "Nasal cavity",
    "Oropharynx, NOS",
    "Overlapping lesion of lip, oral cavity and pharynx",
    "Palate, NOS",
    "Pharynx, NOS",
    "Posterior wall of oropharynx",
    "Retromolar area",
    "Supraglottis",
    "Tongue, NOS",
    "Tonsil, NOS",
    "Upper Gum",
    "Ventral surface of tongue, NOS",
}
cervix_sites = {"Cervix uteri"}
eso_sites = {
    "Cardia, NOS",
    "Esophagus, NOS",
    "Lower third of esophagus",
    "Middle third of esophagus",
    "Thoracic esophagus",
    "Upper third of esophagus",
}
censor_sites = {
    "Breast, NOS",
    "Bladder, NOS",
}

tcga_annotation["Diagnosis"] = None

# Classify each row based on the tumor site
for index, row in tcga_annotation.iterrows():
    site = str(row["Tumor_site"]).strip()
    if site in cervix_sites:
        tcga_annotation.loc[index, "Diagnosis"] = "CERSQ_CA"
    elif site in nsclc_sites:
        tcga_annotation.loc[index, "Diagnosis"] = "NSCLC_SC"
    elif site in hnsq_sites:
        tcga_annotation.loc[index, "Diagnosis"] = "HNSQ_CA"
    elif site in eso_sites:
        tcga_annotation.loc[index, "Diagnosis"] = "ESO_CA_SQ"
    else:
        tcga_annotation.loc[index, "Censor"] = 1
        print(f"Unmatched tumor site: {site} (index {index} - censored)")

# Removed censored samples
tcga_annotation = tcga_annotation[tcga_annotation["Censor"] == 0]

# Extract useful columns
tcga_annotation = tcga_annotation[columns_dict.values()]

### Step 2: Download and unzip the GEO data

In [ ]:
geo_anno_dir_tar = analysis_dir_scc / "geo_annotations.tar.gz"
geo_anno_dir = geo_anno_dir_tar.with_suffix("").with_suffix("")

# Check if the GEO annotation tar file exists and extract
if not geo_anno_dir.exists():
    print("Setting up GEO annotation directory...")
    # TODO download annotation
    extract_tar(geo_anno_dir_tar, analysis_dir_scc)
    print("Setting up GEO annotation directory done.")

# Download the IDAT files.
download_from_geo_and_untar(analysis_dir_scc, datasets[tumor_site]["geo_ids"])


# Download the annotation spreadsheet.
def merge_csv(dir_path):
    """Reads all CSV files merges them."""
    dir_path = Path(dir_path)
    merged_df = pd.DataFrame()
    for csv_file in dir_path.glob("*.csv"):
        print(f"Reading {csv_file}")
        data_frame = pd.read_csv(csv_file)
        merged_df = pd.concat([merged_df, data_frame], ignore_index=True)
    return merged_df


geo_annotation = merge_csv(geo_anno_dir)

### Step 3: Construct the annotation file of all data.
Join the TCGA and GEO annotation files

In [ ]:
if (csv_path := analysis_dir_scc / f"{tumor_site}-annotation.csv").exists():
    print("Merged annotation file allready exists.")
else:
    anno_df = pd.concat([geo_annotation, tcga_annotation], ignore_index=True)
    anno_df.to_csv(csv_path, index=False)
    print("Merged annotation file created.")

### Create the Methylation Analysis Object

The `MethylAnalysis` object serves as the main interface for performing DNA
methylation analysis. Key parameters such as the directory structure, number
of CpG sites, and UMAP settings are configured here.

In [ ]:
analysis_scc = MethylAnalysis(
    analysis_dir=analysis_dir_scc,
    reference_dir=reference_dir,
    output_dir=output_dir,
    test_dir=test_dir_scc,
    n_cpgs=25000,
    load_full_betas=True,
    overlap=False,
    cpg_blacklist=blacklist,
    debug=True,
    do_seg=True,
    umap_parms={
        "n_neighbors": 8,
        "metric": "manhattan",
        "min_dist": 0.3,
    },
)

### Load Beta Values

Reads and processes beta values from the provided dataset. This step can also
be performed interactively within the GUI.

In [ ]:
analysis_scc.set_betas()

### Generate UMAP Plot

Set the columns used for coloring the UMAP plot before initiating the
dimensionality reduction process. The UMAP algorithm produces a visual
representation of the sample clusters, which is stored as a Plotly object in
`analysis_scc.umap_plot`.

In [ ]:
# Calculate UMAP
analysis_scc.idat_handler.selected_columns = ["Diagnosis"]
analysis_scc.make_umap()

In [ ]:
# Show the results
print(analysis_scc.umap_df)
output_path = output_dir / f"{analysis_dir_scc.name}-umap_plot.jpg"
analysis_scc.umap_plot.write_image(
    output_path,
    format="jpg",
    width=2000,
    height=1000,
    scale=2,
)
IPImage(filename=output_path)

### Launch the Analysis GUI

Initializes an interactive GUI for further exploration of the methylation
data.

**Note**: This step is only supported in local environments (not in
cloud-based platforms like Google Colab or Binder).

In [ ]:
analysis_scc.run_app(open_tab=True)

### Generate and Save CNV Plot

Creates a copy number variation (CNV) plot for a specified sample and saves
the output as a high-resolution image.

In [ ]:
# Save CNV example
analysis_scc.make_cnv_plot("364f7953-d0af-4929-8491-7b5e94d488aa_noid")
cnv_plot = analysis_scc.cnv_plot
cnv_plot.update_layout(
    yaxis_range=[-1.1, 1.1],
    font={"size": FONTSIZE},
    margin={"t": 50},
)
output_path = output_dir / f"{analysis_dir_scc.name}-cnv_plot.jpg"
cnv_plot.write_image(
    output_path,
    format="jpg",
    width=2000,
    height=1000,
    scale=2,
)
IPImage(filename=output_path)

### Generate CNV Summary Plots

In addition to individual CNV plots, this step computes summary plots to
highlight genomic alterations across multiple samples.

**Note**:
Generating all copy number variation (CNV) plots can be resource-intensive.
The process may take up to 30 minutes, depending on the computational
resources available.

In [ ]:
analysis_scc.precompute_cnvs()
cn_summary_path_scc = calculate_cn_summary(analysis_scc, "Diagnosis")

In [ ]:
IPImage(filename=cn_summary_path_scc)

### Supervised Classifier Validation

The next step involves validating various supervised classification
algorithms to evaluate their performance on the dataset. This process helps
identify the most accurate model for methylation-based classification.

**Note**:
Training can be resource-intensive. The process may take up to 10 minutes,
depending on the computational resources available.

In [ ]:
# Train supervised classifiers
ids = analysis_scc.idat_handler.ids
clf_out_scc = analysis_scc.classify(
    ids=ids,
    clf_list=[
        "none-kbest-et",
        "none-kbest-lr",
        "none-kbest-rf",
        "none-kbest-svc_rbf",
        "none-pca-lr",
        "none-pca-et",
        "none-none-knn",
    ],
)

In [ ]:
# Print reports for all classifier for the first sample
for clf_result in clf_out_scc:
    print(clf_result.reports[0])

# Identify and display the best classifier
best_clf_scc = max(
    clf_out_scc, key=lambda result: np.mean(result.metrics["accuracy_scores"])
)
print("Most accurate classifier:")
print(best_clf_scc.reports[0])



# 4. Appendix

The GEO annotation files were downloaded and manually curated. Below is the
code used to download the GEO datasets and save their associated metadata:

In [1]:
%%bash
pip install geoparse

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [ ]:
import GEOparse
import pandas as pd

geo_numbers = datasets["scc"]["geo_ids"] + datasets["scc_test"]["geo_ids"]

# Download and save metadata for each GEO series
for geo_nr in geo_numbers:
    gse = GEOparse.get_GEO(geo=geo_nr, destdir=geo_anno_dir)
    metadata = gse.phenotype_data
    print(metadata.head())
    metadata.to_csv(geo_anno_dir / f"{geo_nr}_raw_metadata.csv", index=True)

for file in geo_anno_dir.glob("*.soft.gz"):
    print(f"Removing file: {file}")
    file.unlink()